In [0]:
from pyspark.sql.avro.functions import from_avro
from pyspark.sql.functions import col, expr,when,from_unixtime,to_json
import requests
import json
from requests.auth import HTTPBasicAuth
from pyspark.sql import SparkSession


In [0]:

# Schema Registry Credentials
schema_key = "HEGTSVOYDEWE55DU"
schema_value = "7LtLWK9GaUaoxzCZDKkfjNtJaYY6YbnkwPMcnGk/vnPwTF9OHhDEMJfHR350Q00p"
schema_registry_url = "https://psrc-kg7rp.westus2.azure.confluent.cloud"

# Schema Registry Authentication
schema_registry_options = {
    "schema.registry.url": schema_registry_url,
    "basic.auth.credentials.source": "USER_INFO",
    "schema.registry.basic.auth.user.info": f"{schema_key}:{schema_value}"
}


In [0]:
# Kafka Connection Details
kafka_bootstrap_servers = "pkc-921jm.us-east-2.aws.confluent.cloud:9092"
kafka_security_protocol = "SASL_SSL"
kafka_sasl_mechanism = "PLAIN"
kafka_username = "RWRAIJMUJXJEGQS4"
kafka_password = "3CJveQ6kNaSnhF3dncT/15SILxAz7bDXGeZl6iZjcey5Cf8mSqY/JmMfdIryDQSN"
kafka_topics = "ecommerce.ecommerce.orders,ecommerce.ecommerce.customers,ecommerce.ecommerce.products"

kafka_options = {
    "kafka.bootstrap.servers": kafka_bootstrap_servers,
    "subscribe": kafka_topics,
    "startingOffsets": "earliest",
    "kafka.security.protocol": "SASL_SSL",
    "kafka.sasl.mechanism": "PLAIN",
    "kafka.sasl.jaas.config": f'org.apache.kafka.common.security.plain.PlainLoginModule required username="{kafka_username}" password="{kafka_password}";',
    "failOnDataLoss": "false"
}



# Schema Registry Credentials
schema_key = "HEGTSVOYDEWE55DU"
schema_value = "7LtLWK9GaUaoxzCZDKkfjNtJaYY6YbnkwPMcnGk/vnPwTF9OHhDEMJfHR350Q00p"
schema_registry_url = "https://psrc-kg7rp.westus2.azure.confluent.cloud"


schema_registry_options = {
    "schema.registry.url": schema_registry_url,
    "basic.auth.credentials.source": "USER_INFO",
    "schema.registry.basic.auth.user.info": f"{schema_key}:{schema_value}"
}



In [0]:
schema_subjects = ["ecommerce.ecommerce.orders-value", "ecommerce.ecommerce.customers-value", "ecommerce.ecommerce.products-value"]
schema_registry_auth = HTTPBasicAuth(schema_key, schema_value)
response = requests.get(f"{schema_registry_url}/subjects", auth=schema_registry_auth)
print(response.json())


In [0]:

schemas={}
schema_registry_auth = HTTPBasicAuth(schema_key, schema_value)
for subject in schema_subjects:
    response = requests.get(f"{schema_registry_url}/subjects/{subject}/versions/latest", auth=schema_registry_auth)
    if response.status_code == 200:
         schema_str=response.json()["schema"]
         
         schemas[subject]=schema_str
    else:
        print(f"Error getting schema for subject {subject}, status code {response.status_code}")
print(schemas.keys())

In [0]:
#check connection
df_kafka = spark.readStream.format("kafka").options(**kafka_options).load()
df_kafka.printSchema()

In [0]:
# strip first 5 bytes for proper avro deserialization
df_kafka_clean = df_kafka.withColumn(
    "avro_value", expr("substring(value, 6, length(value)-5)")
).select(
    col("key").cast("string"),  
    col("avro_value"),         
    col("topic"), col("partition"),
    col("offset"), col("timestamp"), col("timestampType")
)

In [0]:
# process oreders topic
df_orders=df_kafka_clean.filter(col("topic")=="ecommerce.ecommerce.orders").select(col('key').cast('string'),from_avro('avro_value',schemas['ecommerce.ecommerce.orders-value']).alias('value_decoded'),col("partition"),col("offset"),col("timestamp"),col("timestampType"))
df_orders_flattened = df_orders.select(
    col("key").cast('string').alias("key"),
    to_json(col("value_decoded.before")).alias("before"),
    to_json(col("value_decoded.after")).alias("after"),
    col("value_decoded.op").alias("op"),
    from_unixtime(col("value_decoded.ts_ms")/1000).alias("ts_ms"),
    to_json(col("value_decoded.transaction")).alias("transaction"),
    to_json(col("value_decoded.source")).alias("source"),
    col("partition"),
    col("offset"),
    col("timestamp"),
    col("timestampType")
)
df_orders_flattened.writeStream.format("delta").option("checkpointLocation", "dbfs:/checkpoints/ecommerce_cdc/bronze_orders").outputMode("append").start("gs://ecommerce_cdc/bronze/orders")

In [0]:
# process customers topic
df_customers=df_kafka_clean.filter(col("topic")=="ecommerce.ecommerce.customers").select(col('key').cast('string'),from_avro('avro_value',schemas['ecommerce.ecommerce.customers-value']).alias('value_decoded'),col("partition"),col("offset"),col("timestamp"),col("timestampType"))
df_customers_flattened = df_customers.select(
    col("key").cast('string').alias("key"),
    to_json(col("value_decoded.before")).alias("before"),
    to_json(col("value_decoded.after")).alias("after"),
    col("value_decoded.op").alias("op"),
    from_unixtime(col("value_decoded.ts_ms")/1000).alias("ts_ms"),
    to_json(col("value_decoded.transaction")).alias("transaction"),
    to_json(col("value_decoded.source")).alias("source"),
    col("partition"),
    col("offset"),
    col("timestamp"),
    col("timestampType")
    )
df_orders_flattened.writeStream.format("delta").option("checkpointLocation", "dbfs:/checkpoints/ecommerce_cdc/bronze_customers").outputMode("append").start("gs://ecommerce_cdc/bronze/customers")

In [0]:
# process products topic
df_products=df_kafka_clean.filter(col("topic")=="ecommerce.ecommerce.products").select(col('key').cast('string'),from_avro('avro_value',schemas['ecommerce.ecommerce.products-value']).alias('value_decoded'),col("partition"),col("offset"),col("timestamp"),col("timestampType"))
df_products_flattened = df_products.select(
    col("key").cast('string').alias("key"),
    to_json(col("value_decoded.before")).alias("before"),
    to_json(col("value_decoded.after")).alias("after"),
    col("value_decoded.op").alias("op"),
    from_unixtime(col("value_decoded.ts_ms")/1000).alias("ts_ms"),
    to_json(col("value_decoded.transaction")).alias("transaction"),
    to_json(col("value_decoded.source")).alias("source"),
    col("partition"),
    col("offset"),
    col("timestamp"),
    col("timestampType")
    )
df_products_flattened.writeStream.format("delta").option("checkpointLocation", "dbfs:/checkpoints/ecommerce_cdc/bronze_products").outputMode("append").start("gs://ecommerce_cdc/bronze/products")